In [1]:
import os,sys
import numpy as np
from collections import namedtuple
import tqdm
import glob
import math
import random
import inspect
import os.path as osp
from pathlib import Path
import itertools
from itertools import chain
import numpy as np
import pandas as pd
import multiprocessing
import h5py
import matplotlib.pyplot as plt
import numpy as np
import sys, os
from importlib import reload

import torch
import torch.nn as nn
from torch.utils.data import random_split
from torch_geometric.data import Data, DataLoader, DataListLoader
from torch_geometric.nn import EdgeConv, global_mean_pool, DataParallel
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import Data,Dataset
from torch_scatter import scatter_mean, scatter
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_geometric.nn import MetaLayer, EdgeConv, global_mean_pool, DynamicEdgeConv


In [2]:
#Data Samples
DATA_PATH = '/eos/cms/store/group/phys_b2g/CASE/h5_files/full_run2/BB_UL_MC_small_v2/'

TRAIN_NAME = 'BB_batch0.h5'
filename_bg = DATA_PATH + TRAIN_NAME 
batch_size = 128
train_set_size = int((5*10e3//batch_size)*batch_size)
#file_bg = h5py.File(filename_bg, 'r') 

In [74]:
deta_jj = 1.4
jPt = 400

def xyze_to_eppt(constituents):
    ''' converts an array [N x 100, 4] of particles
from px, py, pz, E to eta, phi, pt (mass omitted)
    '''
    PX, PY, PZ, E = range(4)
    pt = np.sqrt(np.float_power(constituents[:,PX], 2) + np.float_power(constituents[:,PY], 2), dtype='float32') # numpy.float16 dtype -> float power to avoid overflow
    eta = np.arcsinh(np.divide(constituents[:,PZ], pt, out=np.zeros_like(pt), where=pt!=0.), dtype='float32')
    phi = np.arctan2(constituents[:,PY], constituents[:,PX], dtype='float32')

    return np.stack([pt, eta, phi], axis=1)

side = True
to_train = True

datas = []
for i_e in range(1000):
    if to_train: 
        if file_bg['truth_label'][i_e]!=0 : #train only on QCD
            continue 
    if side :
        if not (file_bg["jet_kinematics"][i_e,1] > deta_jj):
            continue
    else : 
        if not (file_bg["jet_kinematics"][i_e,1] < deta_jj):
            continue
    for i_j in range(2): #each event has 2 jets
        pf_cands = np.array(file_bg["jet{}_PFCands".format(i_j+1)][i_e])
        pf_pt_eta_phi = xyze_to_eppt(pf_cands)
        n_particles = int(np.sum(pf_pt_eta_phi[:,0]!=0)) #if pt!=0
        particles = np.zeros((n_particles, 7)) #px,py,pz,E, pt, eta, phi = 7
        #particles = np.dstack((pf_cands[0:n_particles,:],np.array(pf_pt_eta_phi[0:n_particles,:])))
        particles = np.hstack((pf_cands[0:n_particles,:],np.array(pf_pt_eta_phi[0:n_particles,:])))
        pairs = np.stack([[m, n] for (m, n) in itertools.product(range(n_particles),range(n_particles)) if m!=n])
        edge_index = torch.tensor(pairs, dtype=torch.long)
        edge_index=edge_index.t().contiguous()
        # save particles as node attributes and target
        x = torch.tensor(particles, dtype=torch.float)
        data = Data(x=x, edge_index=edge_index)
        datas.append([data])
datas = sum(datas,[])

In [68]:
def get_present_constit(x,n):
    return x[0:n,:] 

def concat_features(feats_1,feats_2):
    return np.hstack((feats_1[:,:],feats_2[:,:]))

class GraphDataset(Dataset):  ####pytorch Dataset
    def __init__(self, root, transform=None, pre_transform=None,
                 n_events=-1,n_jets=10e3, side_reg=1, features='xyzeptep',n_proc=1):
        """
        Initialize parameters of graph dataset
        Args:
            root (str): dir path
            n_events (int): how many events to process (-1=all in a file (there is a max))
            n_jets (int) : how many total jets to use
            side_reg (bool):true or false, side region for training, otherwise for testing on signal 
            n_proc (int): number of processes to split into
            features (str): (px, py, pz) or relative (pt, eta, phi)
        """
        max_events = int(1.1e6)
        self.n_events = max_events if n_events==-1 else n_events
        self.n_jets = n_jets 
        self.side_reg = side_reg
        self.n_proc = n_proc
        self.chunk_size = self.n_events // self.n_proc
        self.features = features
        self.dEtaJJ = 1.4
        self.jPt = 400
        self.jet_kin_names = ['mJJ', 'DeltaEtaJJ', 'j1Pt', 'j1Eta', 'j1Phi',\
                                        'j1M', 'j2Pt', 'j2Eta', 'j2Phi', 'j2M', 'j3Pt', 'j3Eta', 'j3Phi', 'j3M']
        self.pf_kin_names = ['px','py','pz','E']
        
        super(GraphDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        self.input_files = sorted(glob.glob(self.raw_dir+'/*.root'))
        return [f.split('/')[-1] for f in self.input_files]

    def __len__(self):
      #  return len(self.n_jets) 
        return 50
  
    def xyze_to_ptep(self,constituents):
        ''' converts an array [N x 100, 4] of particles
from px, py, pz, E to eta, phi, pt (mass omitted)
    '''
        PX = self.pf_kin_names.index('px')
        PY = self.pf_kin_names.index('py')
        PZ = self.pf_kin_names.index('pz')
        E = self.pf_kin_names.index('E')
        pt = np.sqrt(np.float_power(constituents[:,:,PX], 2) + np.float_power(constituents[:,:,PY], 2), dtype='float32') # numpy.float16 dtype -> float power to avoid overflow
        eta = np.arcsinh(np.divide(constituents[:,:,PZ], pt, out=np.zeros_like(pt), where=pt!=0.), dtype='float32')
        phi = np.arctan2(constituents[:,:,PY], constituents[:,:,PX], dtype='float32')
        return np.stack([pt, eta, phi], axis=2)



    def read_events(self,idx):
        
        #Data Samples
        DATA_PATH = '/eos/cms/store/group/phys_b2g/CASE/h5_files/full_run2/BB_UL_MC_small_v2/'
        TRAIN_NAME = 'BB_batch0.h5'
        filename_bg = DATA_PATH + TRAIN_NAME 
        in_file = h5py.File(filename_bg, 'r') 
        jet_kin = np.array(in_file["jet_kinematics"])[0:1000]
        truth = np.array(in_file["truth_label"])[0:1000]

        j1Pt_mask = (jet_kin[:,self.jet_kin_names.index('j1Pt')] > self.jPt)
        j2Pt_mask = (jet_kin[:,self.jet_kin_names.index('j2Pt')] > self.jPt)
        full_mask = j1Pt_mask & j2Pt_mask
        if self.side_reg : 
            full_mask = full_mask & (jet_kin[:,self.jet_kin_names.index('DeltaEtaJJ')] > self.dEtaJJ)
        else : 
            full_mask = full_mask & (jet_kin[:,self.jet_kin_names.index('DeltaEtaJJ')] < self.dEtaJJ)

        #Apply mask on jet kinematics, truth and pf cands
        jet_kin = jet_kin[full_mask]
        truth = truth[full_mask]
        jet_const = [np.array(in_file["jet1_PFCands"][0:1000])[full_mask],np.array(in_file["jet2_PFCands"][0:1000])[full_mask]]
                

        pf_out_list = []
        jet_prop_list = []

        for i_j in range(2): #each event has 2 jets
            pf_xyze = jet_const[i_j]
            pf_ptep = self.xyze_to_ptep(pf_xyze)
            n_particles = np.sum(pf_xyze[:,:,self.pf_kin_names.index('E')]!=0,axis=1) #E is 3rd 
            pf_xyze_out = list(map(get_present_constit,pf_xyze,n_particles))
            pf_ptep_out = list(map(get_present_constit,pf_ptep,n_particles))
            pf_tot_out = list(map(concat_features,pf_xyze_out,pf_ptep_out))
            pf_out_list.append(pf_tot_out)

            n_jet_feats = 6
            jet_prop = np.zeros((len(pf_tot_out),n_jet_feats))
            jet_prop[:,0] = n_particles
            for i_f,f_name in enumerate('M,Pt,Eta,Phi'.split(',')):
                jet_prop[:,i_f+1] = jet_kin[:,self.jet_kin_names.index('j{}{}'.format(i_j+1,f_name))]
            jet_prop[:,n_jet_feats-1] = truth[:,0]
            jet_prop_list.append(jet_prop)
            
        #return list of pf particles, and list of global jet properties
        return sum(pf_out_list, []),np.vstack((jet_prop_list[0],jet_prop_list[1]))      
                 

    def get(self,idx):
        '''Yields one data graph'''
        pf_cands, jet_prop = self.read_events(idx)   
        
        i_evt = idx
        #for i_evt in range(len(pf_cands)):
        n_particles = pf_cands[i_evt].shape[0]
        print('event : ',i_evt,' n particle : ',n_particles)
        pairs = np.stack([[m, n] for (m, n) in itertools.product(range(n_particles),range(n_particles)) if m!=n])
        edge_index = torch.tensor(pairs, dtype=torch.long)
        edge_index=edge_index.t().contiguous()
        # save particles as node attributes and target
        x = torch.tensor(pf_cands[i_evt], dtype=torch.float)
        u = torch.tensor(jet_prop[i_evt,:], dtype=torch.float)
        data = Data(x=x, edge_index=edge_index,u=torch.unsqueeze(u, 0))
        print('data.x.n_particles ',n_particles)
        return data
        
data_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/'
dataset = GraphDataset(root=data_dir)
    
validation_split = 0.2
dataset_size = len(dataset)
indices = list(range(dataset_size))
if dataset_size > 2:
    split = int(np.floor(validation_split * dataset_size))
else: 
    split = 1
print(dataset_size,split)
random_seed= 1001

train_subset, val_subset = torch.utils.data.random_split(dataset, [dataset_size - split, split],
                                                             generator=torch.Generator().manual_seed(random_seed))
print("train subset dim:", len(train_subset))
print("validation subset dim", len(val_subset))
dataloaders = {
    'train':  DataLoader(train_subset, batch_size=5, shuffle=True),
    'val':   DataLoader(val_subset, batch_size=5, shuffle=True)
    }
print("train_dataloader dim:", len(dataloaders['train']))
print("val dataloader dim:", len(dataloaders['val']))


50 10
train subset dim: 40
validation subset dim 10
train_dataloader dim: 8
val dataloader dim: 2


In [69]:
dataset.get(0).u

getting data
event :  0  n particle :  46
data.x.n_particles  46


tensor([[ 46.0000,  14.0078, 636.5000,  -1.6787,   1.2065,   0.0000]])

In [16]:
"""
    Model definitions.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import Data
from torch_scatter import scatter_mean, scatter
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_geometric.nn import MetaLayer, EdgeConv, global_mean_pool, DynamicEdgeConv


# GNN AE using EdgeConv (mean aggregation graph operation). Basic GAE model.
class EdgeNet(nn.Module):
    def __init__(self, input_dim=7, output_dim=4, big_dim=32, hidden_dim=2, aggr='mean'):
        super(EdgeNet, self).__init__()
        encoder_nn = nn.Sequential(nn.Linear(2*(input_dim), big_dim),
                               nn.ReLU(),
                               nn.Linear(big_dim, big_dim),
                               nn.ReLU(),
                               nn.Linear(big_dim, hidden_dim),
                               nn.ReLU(),
        )
        
        decoder_nn = nn.Sequential(nn.Linear(2*(hidden_dim), big_dim),
                               nn.ReLU(),
                               nn.Linear(big_dim, big_dim),
                               nn.ReLU(),
                               nn.Linear(big_dim, output_dim)
        )
        
        self.batchnorm = nn.BatchNorm1d(input_dim)

        self.encoder = EdgeConv(nn=encoder_nn,aggr=aggr)
        self.decoder = EdgeConv(nn=decoder_nn,aggr=aggr)

    def forward(self, data):
        x = self.batchnorm(data.x)
        x = self.encoder(x,data.edge_index)
        x = self.decoder(x,data.edge_index)
        return x

In [12]:
def train(model, optimizer, loader, total, batch_size, loss_ftn_obj):
    model.train()

    sum_loss = 0.
    t = tqdm.tqdm(enumerate(loader),total=total/batch_size)
    for i,data in t:
        optimizer.zero_grad()

        batch_loss, batch_output = forward_loss(model, data, loss_ftn_obj, device, multi_gpu=False)
        batch_loss.backward()
        optimizer.step()

        batch_loss = batch_loss.item()
        sum_loss += batch_loss
        t.set_description('train loss = %.7f' % batch_loss)
        t.refresh() # to show immediately the update

    return sum_loss / (i+1)


# helper to perform correct loss
def forward_loss(model, data, loss_ftn_obj, device, multi_gpu=False):
    
    if not multi_gpu:
        data = data.to(device)

    if 'emd_loss' in loss_ftn_obj.name or loss_ftn_obj.name == 'chamfer_loss' or loss_ftn_obj.name == 'hungarian_loss':
        batch_output = model(data)
        if multi_gpu:
            data = Batch.from_data_list(data).to(device)
        y = data.x
        batch = data.batch
        batch_loss = loss_ftn_obj.loss_ftn(batch_output, y, batch)

    elif loss_ftn_obj.name == 'emd_in_forward':
        _, batch_loss = model(data)
        batch_loss = batch_loss.mean()

    elif loss_ftn_obj.name == 'vae_loss':
        batch_output, mu, log_var = model(data)
        y = torch.cat([d.x for d in data]).to(device) if multi_gpu else data.x
        y = y.contiguous()
        batch_loss = loss_ftn_obj.loss_ftn(batch_output, y, mu, log_var)

    else:
        batch_output = model(data)
        y = torch.cat([d.x for d in data]).to(device) if multi_gpu else data.x
        y = y.contiguous()
        batch_loss = loss_ftn_obj.loss_ftn(batch_output, y)

    return batch_loss, batch_output

In [11]:
torch.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
multi_gpu = False #torch.cuda.device_count()>1

In [7]:
class Standardizer:
    def __init__(self):
        self.mean = None
        self.std = None

    def fit(self, data):
        """
        :param data: torch tensor
        """
        self.mean = torch.mean(data, dim=0)
        self.std = torch.std(data, dim=0)

    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data, log_pt=False):
        """
        :param data: torch tensor
        :param log_pt: undo log transformation on pt
        """
        inverse = (data * self.std) + self.mean
        if log_pt:
            inverse[:,0] = (10 ** inverse[:,0]) - 1
        return inverse

def standardize(train_dataset,log_pt=False):
    """
    standardize dataset and return scaler for inversion
    :param train_dataset: list of Data objects
    :param valid_dataset: list of Data objects
    :param test_dataset: list of Data objects
    :param log_pt: log pt before standardization
    :return scaler: sklearn StandardScaler
    """
    train_x = torch.cat([d.x for d in train_dataset])
    if log_pt:
        train_x[:,0] = torch.log(train_x[:,0] + 1)

    scaler = Standardizer()
    scaler.fit(train_x)
    for d in train_dataset:
        d.x[:,:] = scaler.transform(d.x)
    return scaler

In [78]:
#loader = DataLoader(datas, batch_size=128)
#scaler = standardize(datas)


In [17]:
def xyze_to_ptetaphi_torch(y):
    ''' converts an array [N x 100, 4] of particles
from px, py, pz, E to pt,eta, phi
    '''
    PX, PY, PZ, E = range(4)
    pt = torch.sqrt(torch.pow(y[:,PX], 2) + torch.pow(y[:,PY], 2)) 
    eta = torch.asinh(torch.where(pt < 10e-5, torch.zeros_like(pt), torch.div(y[:,PZ], pt)))
    phi = torch.atan2(y[:,PY], y[:,PX])

    relu =  m = nn.ReLU() #inplace=True
    y_E_trimmed = relu(y[:,-1]) #trimming E
    y_pt_trimmed = relu(pt) #trimming pt
    full_y = torch.stack((y[:,0],y[:,1],y[:,2],y_E_trimmed,y_pt_trimmed,eta,phi), dim=1)

    return full_y


class LossFunction:
    def __init__(self, lossname, device=torch.device('cuda:0')):
        loss = getattr(self, lossname)
        self.name = lossname
        self.loss_ftn = loss
        self.device = device
    def mse(self, x, y):
        return F.mse_loss(x, y, reduction='mean')
    
    def mse_coordinates(self, y,x): #for some reason convension is : out,in
        #From px,py,pz,E get pt, eta, phi (do not predict them)
        #x is px,py,pz,E,pt,eta,phi
        #y is px,py,pz,E
        full_y = xyze_to_ptetaphi_torch(y)
        return self.mse(x,full_y)
        

In [18]:
#loss
loss_ftn_obj = LossFunction('mse_coordinates', device=device)

# model
input_dim = 7
output_dim = 4
big_dim = 32
hidden_dim = 2
model = EdgeNet(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)

optimizer = torch.optim.Adam(model.parameters(), lr = 10e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4, threshold=1e-6)

model.to(device)


EdgeNet(
  (batchnorm): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=14, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
    (5): ReLU()
  ))
  (decoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=4, bias=True)
  ))
)

In [70]:
# Training loop
n_epochs = 3
stale_epochs = 0
loss = 999999
train_losses = []
for epoch in range(0, n_epochs):
    #loss = train(model, optimizer, loader, len(datas), 128, loss_ftn_obj)
    loss = train(model, optimizer, dataloaders['train'], 50, 5, loss_ftn_obj)
    train_losses.append(loss)
    print('Epoch: {:02d}, Training Loss:   {:.4f}'.format(epoch, loss))
















  0%|          | 0/10.0 [00:00<?, ?it/s]

getting data
event :  16  n particle :  44
data.x.n_particles  44
getting data
event :  10  n particle :  24
data.x.n_particles  24
getting data
event :  18  n particle :  32
data.x.n_particles  32
getting data
event :  13  n particle :  72
data.x.n_particles  72
getting data

















train loss = 616.6954956:   0%|          | 0/10.0 [00:04<?, ?it/s]














train loss = 616.6954956:   0%|          | 0/10.0 [00:04<?, ?it/s]














train loss = 616.6954956:  10%|█         | 1/10.0 [00:04<00:42,  4.77s/it]

event :  4  n particle :  51
data.x.n_particles  51
getting data
event :  0  n particle :  46
data.x.n_particles  46
getting data
event :  29  n particle :  38
data.x.n_particles  38
getting data
event :  38  n particle :  60
data.x.n_particles  60
getting data
event :  26  n particle :  52
data.x.n_particles  52
getting data

















train loss = 767.9191895:  10%|█         | 1/10.0 [00:08<00:42,  4.77s/it]














train loss = 767.9191895:  10%|█         | 1/10.0 [00:08<00:42,  4.77s/it]














train loss = 767.9191895:  20%|██        | 2/10.0 [00:08<00:36,  4.53s/it]

...

event :  38  n particle :  60
data.x.n_particles  60
getting data
event :  13  n particle :  72
data.x.n_particles  72
getting data
event :  34  n particle :  65
data.x.n_particles  65
getting data
event :  36  n particle :  39
data.x.n_particles  39
getting data
event :  21  n particle :  45
data.x.n_particles  45
getting data

















train loss = 344.7800293:  60%|██████    | 6/10.0 [00:27<00:15,  3.97s/it] 














train loss = 344.7800293:  60%|██████    | 6/10.0 [00:27<00:15,  3.97s/it]














train loss = 344.7800293:  70%|███████   | 7/10.0 [00:27<00:11,  3.97s/it]

event :  47  n particle :  59
data.x.n_particles  59
getting data
event :  23  n particle :  24
data.x.n_particles  24
getting data
event :  4  n particle :  51
data.x.n_particles  51
getting data
event :  32  n particle :  52
data.x.n_particles  52
getting data
event :  49  n particle :  35
data.x.n_particles  35
getting data

















train loss = 1141.5419922:  70%|███████   | 7/10.0 [00:31<00:11,  3.97s/it]














train loss = 1141.5419922:  70%|███████   | 7/10.0 [00:31<00:11,  3.97s/it]














train loss = 1141.5419922:  80%|████████  | 8/10.0 [00:31<00:08,  4.00s/it]

event :  43  n particle :  46
data.x.n_particles  46
Epoch: 02, Training Loss:   704.0702


In [85]:
@torch.no_grad()
def gen_in_out(model, loader, device):
    model.eval()
    input_fts = []
    reco_fts = []

    for t in loader:
        if isinstance(t, list):
            for d in t:
                input_fts.append(d.x)
        else:
            input_fts.append(t.x)
            t.to(device)

        reco_out = model(t)
        if isinstance(reco_out, tuple):
            reco_out = reco_out[0]
        reco_fts.append(reco_out.cpu().detach())

    input_fts = torch.cat(input_fts)
    reco_fts = torch.cat(reco_fts)
    return input_fts, reco_fts

def plot_reco_for_loader(model, loader, device, scaler, inverse_scale, model_fname, save_dir, feature_format):
    input_fts, reco_fts = gen_in_out(model, loader, device)
    if inverse_scale:
        input_fts = scaler.inverse_transform(input_fts)
        reco_fts = scaler.inverse_transform(reco_fts)
    plot_reco_difference(input_fts, reco_fts, model_fname, save_dir, feature_format)

    
def plot_reco_difference(input_fts, reco_fts, model_fname, save_path, feature='hadronic'):
    """
    Plot the difference between the autoencoder's reconstruction and the original input
    Args:
        input_fts (numpy array): the original features of the particles
        reco_fts (numpy array): the reconstructed features
        model_fname (str): name of saved model
    """
    
    if isinstance(input_fts, torch.Tensor):
        input_fts = input_fts.numpy()
    if isinstance(reco_fts, torch.Tensor):
        if feature == 'all':
            reco_fts = xyze_to_ptetaphi_torch(reco_fts)
        reco_fts = reco_fts.numpy()

        
    Path(save_path).mkdir(parents=True, exist_ok=True)
  #  label = ['$p_x~[GeV]$', '$p_y~[GeV]$', '$p_z~[GeV]$']
   # feat = ['px', 'py', 'pz']
    label = ['$p_x~[GeV]$', '$p_y~[GeV]$', '$p_z~[GeV]$']
    feat = ['px', 'py', 'pz']
    if feature == 'hadronic':# or 'standardized':
        label = ['$p_T$', '$eta$', '$phi$']
        feat = ['pt', 'eta', 'phi']
        
    if feature == 'all':# or 'standardized':
        label = ['$p_x~[GeV]$', '$p_y~[GeV]$', '$p_z~[GeV]$', '$E~[GeV]$','$p_T$', '$eta$', '$phi$']
        feat = ['px', 'py', 'pz','E','pt', 'eta', 'phi']
        
    # make a separate plot for each feature
    for i in range(input_fts.shape[1]):
        #plt.style.use(hep.style.CMS)
        plt.figure(figsize=(10,8))
        if feature == 'cartesian':
            bins = np.linspace(-20, 20, 101)
            if i == 3:  # different bin size for E momentum
                bins = np.linspace(-5, 35, 101)
        elif feature == 'hadronic':
            bins = np.linspace(-2, 2, 101)
            if i == 0:  # different bin size for pt rel
                bins = np.linspace(-0.05, 0.1, 101)
        elif feature == 'all':
            bins = np.linspace(-20, 20, 101)
            if i > 3:  # different bin size for hadronic coord
                bins = np.linspace(-2, 2, 101)
            if i == 3:  # different bin size for E momentum
                bins = np.linspace(-5, 35, 101)
            if i == 4:  # different bin size for pt rel
                bins = np.linspace(-2, 10, 101)
        else:
            bins = np.linspace(-1, 1, 101)
        plt.ticklabel_format(useMathText=True)
        plt.hist(input_fts[:,i], bins=bins, alpha=0.5, label='Input', histtype='step', lw=5)
        plt.hist(reco_fts[:,i], bins=bins, alpha=0.5, label='Output', histtype='step', lw=5)
        plt.legend(title='QCD dataset', fontsize='x-large')
        plt.xlabel(label[i], fontsize='x-large')
        plt.ylabel('Particles', fontsize='x-large')
        plt.tight_layout()
        plt.savefig(osp.join(save_path, feat[i] + '.pdf'))
        plt.close()
    

In [86]:
inverse_standardization = False
save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/'
plot_reco_for_loader(model, loader, device, scaler, inverse_standardization, 'test_train', osp.join(save_dir, 'reconstruction_post_train', 'train'), 'all')
